# FER2013 Dataset Model Training

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight

2025-06-18 21:39:55.595239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750282795.617666    1126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750282795.624460    1126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data Preprocessing

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range =  0.1,
    validation_split=0.3
)

batch_size = 64

train_generator = datagen.flow_from_directory(
    '/kaggle/input/facial-expressions-dataset/images/train', #the dataset was loaded and trained on kaggle
    target_size=(224, 224),
    color_mode='rgb', #3channel
    batch_size=batch_size,
    class_mode='categorical', #onehot
    subset='training'
)


val_generator = datagen.flow_from_directory(
    '/kaggle/input/facial-expressions-dataset/images/validation',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 20179 images belonging to 7 classes.
Found 2117 images belonging to 7 classes.


## EfficientNet Transfer Learning Model

Customised with very heavy regularisation to prevent overfitting

In [ ]:
base_model = EfficientNetV2S(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
#EfficientNet model trained with imagenet weights with classification head removed

base_model.trainable = True #all weights are editable

model = Sequential([
    base_model,
    GlobalAveragePooling2D(), #average of the 1280 7x7 feature maps

    Dense(256, activation='relu', kernel_regularizer=l2(0.001)), #regulariser = 0.001 * sum of lambas squared - added to loss term
    Dropout(0.35), # 35% neuron outputs dropped, remaining scaled up by 1/(1-0.35) only during train

    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.35),

    Dense(7, activation='softmax')  # 7 emotions, 7 one hot encoding, categorical crossentropy loss
])


model.compile(
    optimizer='adam', #momentum + parameter wise scaling
    loss= tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2), #reduces overconfidence, allows hedging, 1=0.8, 0=0.2/6
    metrics=['accuracy']
)

model.summary()


I0000 00:00:1750282800.135607    1126 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-s (Functional)        │ (None, 7, 7, 1280)          │      20,331,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,693,095 (78.94 MB)

 Trainable params: 20,539,223 (78.35 MB)

 Non-trainable params: 153,872 (601.06 KB)

## Model Training

Model keeps saving as best weights are found. KeyBoard interrupt when there is a growing difference between accuracy and val accuracy

In [ ]:
early_stop = EarlyStopping( #early stopping if val doesnt improve past 4 epochs
    monitor='val_loss', 
    patience=4,
    restore_best_weights=True)

model_checkpoint = ModelCheckpoint( #continual saving on improvements
    filepath='best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

reduce_lr = ReduceLROnPlateau( #reduces LR when scores plateau to prevent circling around the minimum
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

y_train = train_generator.classes

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

history = model.fit(
    train_generator,
    epochs= 20,
    validation_data = val_generator,
    class_weight = class_weights, #counters class imbalance with penalising loss *= (nsmaples)/(nclasses*samplesofclass)
    callbacks = [early_stop, reduce_lr, model_checkpoint]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1750282919.210197    1189 service.cc:148] XLA service 0x79dfbc003170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750282919.210239    1189 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750282929.643156    1189 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1750282947.026715    1189 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750282947.222704    1189 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750282947.768897    1189 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750282947.9780

158/316 ━━━━━━━━━━━━━━━━━━━━ 1:56 739ms/step - accuracy: 0.2342 - loss: 2.4633

E0000 00:00:1750283121.595239    1186 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750283121.784192    1186 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750283122.191582    1186 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750283122.390723    1186 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750283122.925200    1186 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 932ms/step - accuracy: 0.2839 - loss: 2.3688
Epoch 1: val_accuracy improved from -inf to 0.45489, saving model to best_model.h5
316/316 ━━━━━━━━━━━━━━━━━━━━ 519s 1s/step - accuracy: 0.2842 - loss: 2.3683 - val_accuracy: 0.4549 - val_loss: 1.8623 - learning_rate: 0.0010
Epoch 2/20
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 737ms/step - accuracy: 0.4548 - loss: 1.8790
Epoch 2: val_accuracy improved from 0.45489 to 0.49740, saving model to best_model.h5
316/316 ━━━━━━━━━━━━━━━━━━━━ 259s 819ms/step - accuracy: 0.4549 - loss: 1.8788 - val_accuracy: 0.4974 - val_loss: 1.6644 - learning_rate: 0.0010
Epoch 3/20
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 742ms/step - accuracy: 0.4987 - loss: 1.6840
Epoch 3: val_accuracy improved from 0.49740 to 0.52480, saving model to best_model.h5
316/316 ━━━━━━━━━━━━━━━━━━━━ 261s 824ms/step - accuracy: 0.4987 - loss: 1.6839 - val_accuracy: 0.5248 - val_loss: 1.5750 - learning_rate: 0.0010
Epoch 4/20
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 746ms/step - accu

KeyboardInterrupt: 

## At this point, the model is saved as an .h5 file. The model is stopped from training to prevent too much overfitting